<h1>Segmenting and Clustering Neighborhoods in Toronto<h1>

<h2>Question 1<h2>

In [167]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [168]:
# import the library we use to open URLs
import urllib.request

First of all, we specify which URL/web page we are going to be scraping.

In [169]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050"

In [170]:
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

Then, we import the BeautifulSoup library so we can parse HTML and XML documents and parse the HTML from our URL into the BeautifulSoup parse tree format.

In [171]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page, "lxml")

Through the prettify() command, we can have a look at the structure of our data.

In [172]:
#print(soup.prettify())

We use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable.

In [173]:
all_tables=soup.find_all("table")

We notice that the table we are interested in is marked by the class 'wikitable sortable', so we use this information to save our table as right_table.

In [174]:
right_table=soup.find('table', class_='wikitable sortable')

Since the table consists of three columns, we define A, B and C, where we will save the data contained in the table by column.

In [175]:
A = []
B = []
C = []

We run this loop to exclude empty rows. 

In [176]:
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

We convert our data into a dataframe and use A, B and C as its columns.

In [177]:
df=pd.DataFrame(A,columns=['Postcode'])
df['Borough']=B
df['Neighborhood']=C
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Since we are not interested in the rows containing the value 'Not assigned', we drop them.

In [178]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace = True) 
df.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


We notice that more than one neighborhood can exist in one postal code area, as the following table shows as well.

In [179]:
df.groupby(['Postcode']).count().head()

,Borough,Neighborhood
Postcode,,
M1B,2,2
M1C,3,3
M1E,3,3
M1G,1,1
M1H,1,1


To solve this problem, we create a new dataframe where neighborhoods are grouped by their postal code.

In [180]:
gb = df.groupby(['Postcode', 'Borough'])
result = gb['Neighborhood'].unique()
new_df=pd.DataFrame(result)
new_df.reset_index(inplace=True)
new_df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood , Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae ]


Now we want to remove the square brackets from the elements in the 'Neighborhood' column.

In [181]:
new_df['Neighborhood']=new_df['Neighborhood'].apply(', '.join)
new_df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae\n


Finally, we print the number of rows of our new dataframe.

In [182]:
n_rows = new_df.shape
print('Our new dataframe has', n_rows[0], 'rows.' )

Our new dataframe has 103 rows.


<h2>Question 2<h2>

We import the csv file containing the coordinates of each postal code and convert it to a pandas dataframe.

In [183]:
df_coord = pd.read_csv('https://cocl.us/Geospatial_data') 
print('The dataframe contains',df_coord.shape[0], 'rows.')
df_coord.head()


The dataframe contains 103 rows.


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now we observe two facts:

a. this dataframe contains as many rows as the new_dataframe from question 1

b. in both dataframes, the postal codes are in the same order (from M1B to M9W).

Therefore, we can directly add the columns 'Latitude' and 'Longitude' from this dataframe to the dataframe we used in question 1.

In [184]:
new_df['Latitude'] = df_coord['Latitude'].values 
new_df['Longitude'] = df_coord['Longitude'].values
new_df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae\n,43.773136,-79.239476


<h2> Question 3 <h2>

<h3>3 a. Exploring the dataset<h3>

We use the geopy library to get the latitude and longitude values of Toronto.

In [185]:
from geopy.geocoders import Nominatim
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude)) 

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Now we import folium to create a maps of Toronto's neighborhoods.

In [186]:
! pip install folium
import folium 

Now we're ready to create the map!

In [187]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(new_df['Latitude'], new_df['Longitude'], new_df['Borough'], new_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto 

Let's simplify the above map and segment and cluster only the boroughs that contain the word 'Toronto'. So let's slice the original dataframe and create a new dataframe of the Toronto data.

In [188]:
toronto_data = new_df[new_df['Borough'].str.contains('Toronto')]
toronto_data = toronto_data.reset_index()
toronto_data = toronto_data.drop(columns='index')
toronto_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West\n, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West\n, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District\n,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


As we did with all of Toronto, let's create a map of the boroughs whose names contain the word 'Toronto'.

In [189]:
# create map of Toronto using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them. First, we define the Foursquare Credentials and Version.

In [190]:
CLIENT_ID = 'TLIHYDEOP51XCTHFTDINAAYTONTLGSM0RBE0DDU0340BIT11' # your Foursquare ID
CLIENT_SECRET = 'GAMU2WXV4I3PFWGUWPQ3YEVE2KDHP2YI15TU3AEDLIM1XJ32' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET) 

Your credentails:
CLIENT_ID: TLIHYDEOP51XCTHFTDINAAYTONTLGSM0RBE0DDU0340BIT11
CLIENT_SECRET:GAMU2WXV4I3PFWGUWPQ3YEVE2KDHP2YI15TU3AEDLIM1XJ32


Now we want to explore the first neighborhood in our dataframe. Let's get its name!

In [191]:
toronto_data.iloc[0]['Neighborhood'] 

'The Beaches'

Get the neighborhood's latitude and longitude values.

In [192]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude)) 

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


Now, let's get the top 100 venues that are in The Beaches within a radius of 500 meters.

First, let's create the GET request URL. 

In [193]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=TLIHYDEOP51XCTHFTDINAAYTONTLGSM0RBE0DDU0340BIT11&client_secret=GAMU2WXV4I3PFWGUWPQ3YEVE2KDHP2YI15TU3AEDLIM1XJ32&ll=43.67635739999999,-79.2930312&v=20180605&radius=500&limit=100'

Now we import the necessary libraries for this task.

In [194]:
import json # library to handle JSON files
import requests # library to handle requests


Now we send the GET request and examine the results.

In [195]:
results = requests.get(url).json()
#results 

We know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [196]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name'] 

Now we are ready to clean the json and structure it into a pandas dataframe.

In [197]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe 

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues 

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


And how many venues were returned by Foursquare?

In [198]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0])) 

4 venues were returned by Foursquare.


<h3> 3 b. Exploring neighborhoods in the boroughs containing the word 'Toronto'<h3>

Let's create a function to repeat the same process to all the neighborhoods in the boroughs containing the word 'Toronto'.

In [199]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues) 

Now we will write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [200]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  ) 
#toronto_venues

The Beaches
The Danforth West
, Riverdale
The Beaches West
, India Bazaar
Studio District

Lawrence Park
Davisville North

North Toronto West

Davisville

Moore Park, Summerhill East

Deer Park, Forest Hill SE
, Rathnelly, South Hill, Summerhill West

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson
, Garden District

St. James Town
Berczy Park
Central Bay Street

Adelaide
, King
, Richmond

Harbourfront East
, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel

Roselawn

Forest Hill North, Forest Hill West

The Annex, North Midtown
, Yorkville
Harbord
, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay
, Island airport
, Harbourfront West
, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade

First Canadian Place, Underground city
Christie

Dovercourt Village, Dufferin

Little Portugal, Trinity
Brockton
, Exhibition Place, Parkdale Village


Let's check the size of the resulting dataframe

In [201]:
print(toronto_venues.shape)
toronto_venues.head() 

(1623, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West\n, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


Let's check how many venues were returned for each neighborhood.

In [202]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide\n, King\n, Richmond\n",94,94,94,94,94,94
Berczy Park,57,57,57,57,57,57
"Brockton\n, Exhibition Place, Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern\n,18,18,18,18,18,18
"CN Tower, Bathurst Quay\n, Island airport\n, Harbourfront West\n, King and Spadina, Railway Lands, South Niagara",17,17,17,17,17,17
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street\n,63,63,63,63,63,63
"Chinatown, Grange Park, Kensington Market",55,55,55,55,55,55
Christie\n,17,17,17,17,17,17


Let's find out how many unique categories can be curated from all the returned venues.

In [203]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique()))) 

There are 231 uniques categories.


<h3>3 c. Analyze each neighborhood <h3>

In [204]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-72]] + list(toronto_onehot.columns[:-72])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head() 

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West\n, Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [205]:
toronto_onehot.shape

(1623, 160)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [206]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue
0,"Adelaide\n, King\n, Richmond\n",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.0,0.0,0.010638,0.010638,0.0,0.010638,0.0,0.000000,0.000000,0.0,0.000000,0.021277,0.0,0.010638,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.021277,0.000000,0.010638,0.021277,0.000000,0.0,0.010638,0.010638,0.010638,0.0,0.0,0.053191,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.031915,0.000000,0.095745,0.0,0.0,0.0,0.0,0.010638,0.000000,0.000000,0.021277,0.0,0.000000,0.0,0.021277,0.0,0.000000,0.0,0.010638,0.0,0.031915,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.010638,0.0,0.000000,0.010638,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.010638,0.0,0.000000,0.000000,0.0,0.010638,0.0,0.0,0.010638,0.0,0.010638,0.010638,0.000000,0.000000,0.000000,0.031915,0.010638,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.031915,0.0,0.0,0.010638,0.0,0.0,0.000000,0.0,0.000000,0.010638,0.000000,0.0,0.010638,0.0,0.0,0.010638,0.000000,0.0,0.000000,0.010638,0.0,0.0,0.010638,0.0,0.0,0.0,0.0,0.010638,0.0,0.010638,0.0,0.0,0.000000,0.0
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.017544,0.000000,0.0,0.000000,0.0,0.000000,0.017544,0.0,0.017544,0.035088,0.0,0.000000,0.0,0.017544,0.017544,0.0,0.035088,0.0,0.0,0.017544,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.035088,0.0,0.0,0.0,0.035088,0.0,0.0,0.0,0.000000,0.017544,0.052632,0.070175,0.0,0.0,0.0,0.0,0.000000,0.017544,0.000000,0.017544,0.0,0.000000,0.0,0.017544,0.0,0.017544,0.0,0.000000,0.0,0.000000,0.017544,0.0,0.017544,0.0,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.

Let's confirm the new size.

In [207]:
toronto_grouped.shape

(39, 160)

Let's print each neighborhood along with the top 5 most common venues. 

In [208]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n') 

----Adelaide
, King
, Richmond
----
            venue  freq
0     Coffee Shop  0.10
1            Café  0.05
2  Clothing Store  0.03
3   Deli / Bodega  0.03
4             Gym  0.03


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2   Cheese Shop  0.04
3          Café  0.04
4        Bakery  0.04


----Brockton
, Exhibition Place, Parkdale Village----
            venue  freq
0            Café  0.13
1     Coffee Shop  0.09
2  Breakfast Spot  0.09
3             Bar  0.04
4          Bakery  0.04


----Business Reply Mail Processing Centre 969 Eastern
----
                  venue  freq
0         Garden Center  0.06
1        Farmers Market  0.06
2  Fast Food Restaurant  0.06
3                Garden  0.06
4  Gym / Fitness Center  0.06


----CN Tower, Bathurst Quay
, Island airport
, Harbourfront West
, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3

Let's put that into a pandas dataframe!

First, let's write a function to sort the venues in descending order.

In [209]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues] 

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [210]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head() 

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide\n, King\n, Richmond\n",Coffee Shop,Café,Deli / Bodega,Hotel,Gym,Clothing Store,American Restaurant,Bookstore,Breakfast Spot,Bakery
1,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Café,Beer Bar,Bakery,Japanese Restaurant,Comfort Food Restaurant,Basketball Stadium,Beach
2,"Brockton\n, Exhibition Place, Parkdale Village",Café,Breakfast Spot,Coffee Shop,Burrito Place,Convenience Store,Climbing Gym,Furniture / Home Store,Intersection,Italian Restaurant,Grocery Store
3,Business Reply Mail Processing Centre 969 East...,Gym / Fitness Center,Burrito Place,Comic Shop,Garden,Brewery,Farmers Market,Fast Food Restaurant,Garden Center,Light Rail Station,Auto Workshop
4,"CN Tower, Bathurst Quay\n, Island airport\n, H...",Airport Service,Airport Lounge,Airport Terminal,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Bar,Boat or Ferry,Boutique


<h3>3 d. Cluster neighborhoods<h3>

First, we need to import k-means.

In [211]:
from sklearn.cluster import KMeans

Now let's run k-means to cluster the neighborhoods into 5 clusters.

In [212]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_  

array([4, 4, 4, 0, 0, 4, 4, 4, 0, 4, 4, 4, 3, 4, 4, 0, 4, 2, 0, 4, 4, 0,
       2, 0, 0, 4, 4, 4, 0, 1, 4, 4, 4, 4, 4, 4, 0, 0, 4], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [213]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns! 

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Bus Line,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop,Cocktail Bar,Clothing Store,Climbing Gym
1,M4K,East Toronto,"The Danforth West\n, Riverdale",43.679557,-79.352188,4,Greek Restaurant,Coffee Shop,Italian Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Ice Cream Shop,Bookstore,Brewery,Japanese Restaurant,Indian Restaurant
2,M4L,East Toronto,"The Beaches West\n, India Bazaar",43.668999,-79.315572,0,Fast Food Restaurant,Burrito Place,Movie Theater,Gym,Brewery,Ice Cream Shop,Food & Drink Shop,Italian Restaurant,Fish & Chips Shop,Liquor Store
3,M4M,East Toronto,Studio District\n,43.659526,-79.340923,4,Café,Coffee Shop,Bakery,Gastropub,Brewery,American Restaurant,Diner,Bank,Italian Restaurant,Ice Cream Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Bus Line,Construction & Landscaping,Colombian Restaurant,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop,Cocktail Bar,Clothing Store


Finally, let's visualize the resulting clusters!

In [214]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters 

<h3>3 e. Examine Clusters <h3>

Now, let's examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we will assign a name to each cluster.

<h4>Cluster 1<h4>

In [215]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]] 

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Health Food Store,Bus Line,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop,Cocktail Bar,Clothing Store,Climbing Gym
2,East Toronto,0,Fast Food Restaurant,Burrito Place,Movie Theater,Gym,Brewery,Ice Cream Shop,Food & Drink Shop,Italian Restaurant,Fish & Chips Shop,Liquor Store
8,Central Toronto,0,Music Venue,Climbing Gym,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop,Cocktail Bar,Clothing Store,Church
10,Downtown Toronto,0,Music Venue,Climbing Gym,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop,Cocktail Bar,Clothing Store,Church
25,Downtown Toronto,0,Café,Bakery,Bookstore,Italian Restaurant,Japanese Restaurant,Bar,Chinese Restaurant,Coffee Shop,College Arts Building,College Gym
27,Downtown Toronto,0,Airport Service,Airport Lounge,Airport Terminal,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Bar,Boat or Ferry,Boutique
30,Downtown Toronto,0,Grocery Store,Café,Diner,Candy Store,Italian Restaurant,Baby Store,Coffee Shop,Athletics & Sports,Cocktail Bar,College Gym
31,West Toronto,0,Bakery,Music Venue,Liquor Store,Café,Gym / Fitness Center,Brewery,Gas Station,Bar,Bank,Grocery Store
32,West Toronto,0,Bar,Asian Restaurant,Café,Men's Store,Brewery,Greek Restaurant,Boutique,Beer Store,Cocktail Bar,French Restaurant
34,West Toronto,0,Café,Mexican Restaurant,Music Venue,Bakery,Fast Food Restaurant,Flea Market,Fried Chicken Joint,Furniture / Home Store,Gastropub,Cajun / Creole Restaurant


In cluster 1, among the most common venues in each neighborhood there are gyms, climbing gyms, fitness centers, athletics and sports centers. We will name this cluster as the Fitness Cluster.

<h4>Cluster 2<h4>

In [216]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]] 

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,1,Garden,Music Venue,Donut Shop,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop,Cocktail Bar,Clothing Store


Cluster 2 consists of only one neighborhood. Since its most common venue is a garden, we will name it as the Garden Cluster.

<h4>Cluster 3<h4>

In [217]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]] 

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,2,Bus Line,Construction & Landscaping,Colombian Restaurant,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop,Cocktail Bar,Clothing Store
23,Central Toronto,2,Bus Line,Jewelry Store,Music Venue,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop,Cocktail Bar,Clothing Store


In cluster 3, the most common venue of the two neighborhoods composing the cluster is the bus line. We will name this cluster as the Commuters' Cluster.

<h4>Cluster 4<h4>

In [218]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]] 

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Toronto,3,Dog Run,Department Store,Breakfast Spot,Hotel,Food & Drink Shop,Gym,Dessert Shop,Chinese Restaurant,Coffee Shop,Cocktail Bar


Cluster 4 consists of only one neighborhood. Since its most common venue is a dog run, we will name it as the Dog Run Cluster.

<h4>Cluster 5<h4>

In [219]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]] 

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,4,Greek Restaurant,Coffee Shop,Italian Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Ice Cream Shop,Bookstore,Brewery,Japanese Restaurant,Indian Restaurant
3,East Toronto,4,Café,Coffee Shop,Bakery,Gastropub,Brewery,American Restaurant,Diner,Bank,Italian Restaurant,Ice Cream Shop
6,Central Toronto,4,Coffee Shop,Clothing Store,Furniture / Home Store,Health & Beauty Service,Diner,Dessert Shop,Gym / Fitness Center,Café,Mexican Restaurant,Fast Food Restaurant
7,Central Toronto,4,Dessert Shop,Gym,Café,Coffee Shop,Italian Restaurant,Diner,Bar,Brewery,Gas Station,Indian Restaurant
9,Central Toronto,4,Coffee Shop,Health & Beauty Service,American Restaurant,Bank,Bagel Shop,Light Rail Station,Liquor Store,Fried Chicken Joint,Chinese Restaurant,Chocolate Shop
11,Downtown Toronto,4,Coffee Shop,Italian Restaurant,Café,Bakery,Beer Store,Bank,Japanese Restaurant,Flower Shop,Indian Restaurant,Chinese Restaurant
12,Downtown Toronto,4,Coffee Shop,Japanese Restaurant,Gay Bar,Grocery Store,Mediterranean Restaurant,Hotel,Men's Store,Gastropub,Clothing Store,Beer Bar
13,Downtown Toronto,4,Coffee Shop,Bakery,Café,Breakfast Spot,Gym / Fitness Center,Beer Store,Ice Cream Shop,Hotel,Historic Site,Health Food Store
14,Downtown Toronto,4,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Cosmetics Shop,Italian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Fast Food Restaurant,Lingerie Store
15,Downtown Toronto,4,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Department Store,Moroccan Restaurant,Cosmetics Shop,Italian Restaurant,Clothing Store


In cluster 5, among the four most common venues of each neighborhood there's always a café, a coffee shop or a dessert shop, so we will name this cluster as the Coffee Cluster.